# Fine-Tuning standard con LoRA

#### Configurazioni generali

Installazione delle librerie necessarie.

Importo i moduli necessari.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from collections import Counter
from transformers import BertTokenizer
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score, f1_score
from peft import LoraConfig, get_peft_model
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification

2025-03-26 09:25:32.761608: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 09:25:32.775393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742981132.793028  349448 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742981132.798623  349448 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742981132.811637  349448 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Impostazione del seme casuale per la riproducibilità.

In [2]:
seed_value = 42

os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

# Imposto il seme casuale anche per i calcoli CUDA
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Sentiment140

In [4]:
# Ottenimento del dataset
sent140_dataset = load_dataset("stanfordnlp/sentiment140",trust_remote_code=True)
print(sent140_dataset)

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

sentiment140.py:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 1600000
    })
    test: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 498
    })
})


In [5]:
# Divido i dati in training, validation e test set
sent140_data = sent140_dataset["train"].shuffle(seed=42)

sent140_temp_sentences, sent140_test_sentences, sent140_temp_labels, sent140_test_labels = train_test_split(
                                                sent140_data['text'], 
                                                sent140_data['sentiment'], 
                                                test_size=1024, 
                                                random_state=42,
                                                stratify=sent140_data['sentiment'])

sent140_train_sentences, sent140_val_sentences, sent140_train_labels, sent140_val_labels = train_test_split(
                                                sent140_temp_sentences, 
                                                sent140_temp_labels, 
                                                train_size=20000,
                                                test_size=1000,
                                                random_state=42,
                                                stratify=sent140_temp_labels)

# Trasformazione delle etichette 0 -> 0 e 4->1
sent140_train_labels = [1 if label == 4 else 0 for label in sent140_train_labels]
sent140_val_labels = [1 if label == 4 else 0 for label in sent140_val_labels]
sent140_test_labels = [1 if label == 4 else 0 for label in sent140_test_labels]


print("Dimensioni dei set:")
print(f"Train: {len(sent140_train_sentences)}")
print(f"Validation: {len(sent140_val_sentences)}")
print(f"Test: {len(sent140_test_sentences)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(sent140_train_labels)}")
print(f"Validation: {Counter(sent140_val_labels)}")
print(f"Test: {Counter(sent140_test_labels)}")

Dimensioni dei set:
Train: 20000
Validation: 1000
Test: 1024

Distribuzione delle etichette:
Train: Counter({0: 10000, 1: 10000})
Validation: Counter({1: 500, 0: 500})
Test: Counter({1: 512, 0: 512})


In [10]:
class ClassificationDataset(Dataset):

    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self,index):
        sentence = self.sentences[index]
        label = self.labels[index]
        
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=True,
            padding="max_length",
            return_attention_mask=True,
            return_tensors='pt')
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding["token_type_ids"].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
            }

In [ ]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
sent140_training_data = ClassificationDataset(
                           sentences = sent140_train_sentences,
                           labels = sent140_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

sent140_validation_data = ClassificationDataset(
                           sentences = sent140_val_sentences,
                           labels = sent140_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

sent140_test_data = ClassificationDataset(
                           sentences = sent140_test_sentences,
                           labels = sent140_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [4]:
from tqdm import tqdm
import time
import torch
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn
from codecarbon import EmissionsTracker


# Funzione di training e valutazione
def train_and_evaluate_model(model, dataset, train_loader, val_loader, optimizer, scheduler, device, epochs=10, patience=3):

    os.makedirs("carbon_emissions", exist_ok=True)
    tracker = EmissionsTracker(output_dir="carbon_emissions", output_file="emissions.csv")  
    tracker.start()  

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    best_accuracy = 0
    best_loss = float('inf')
    patience_counter = 0  

    start_time = time.time()

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Training
        train_loss, train_acc = train_model(model, train_loader, optimizer, scheduler, device)
        
        # Valutazione
        val_loss, val_acc, val_f1 = eval_model(model, val_loader, device)
        
        # Salvataggio del modello migliore
        if val_acc > best_accuracy:
            torch.save(model.state_dict(),  f"{dataset}_best_model_state.bin")
            best_accuracy = val_acc

        # Salvataggio delle metriche
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0 
        else:
            patience_counter += 1
            print(f"La loss sul validation set non è migliorata per {patience_counter} epoche.")

        if patience_counter >= patience:
            print(f"Early stopping attivato dopo {patience_counter} epoche senza miglioramenti")
            break

    end_time = time.time()
    total_training_time = end_time - start_time

    emissions = tracker.stop()
    print(f"\nEmissioni CO₂ totali: {emissions:.4f} kg")  

    return history, total_training_time, emissions

In [5]:
# Funzione di training
def train_model(model, data_loader, optimizer, scheduler, device):

    model = model.train()

    total_loss = 0
    all_preds = []
    all_labels = []

    loop = tqdm(data_loader, desc=f"Training  ", leave=True)

    for batch in loop:

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # --- Forward pass ---
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels 
        )

        loss = outputs.loss  
        logits = outputs.logits  

        # --- Backward pass ---
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=1)  # Predizioni multiclasse

        all_preds.extend(preds.detach().cpu().numpy())
        all_labels.extend(labels.detach().cpu().numpy())

        loop.set_postfix(loss=total_loss / (loop.n + 1), accuracy=accuracy_score(all_labels, all_preds))

    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)

    return avg_loss, accuracy

In [6]:
# Funzione di valutazione
def eval_model(model, data_loader, device):

    model = model.eval()

    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        
        loop = tqdm(data_loader, desc=f"Evaluating", leave=True)
        for batch in loop:
            
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(labels.detach().cpu().numpy())

            loop.set_postfix(loss=total_loss / (loop.n + 1), accuracy=accuracy_score(all_labels, all_preds))

    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")
    
    return avg_loss, accuracy, f1  

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device)

In [ ]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

for name, param in lora_model.named_parameters():
    if "classifier" in name:
        print(f"{name}: requires_grad = {param.requires_grad}")

In [ ]:
# Parametri principali
learning_rate = 5e-4
EPOCHS = 5
BATCH_SIZE = 32

# Creo i DataLoader
sent140_train_loader = DataLoader(sent140_training_data, batch_size=BATCH_SIZE, shuffle=True)
sent140_val_loader = DataLoader(sent140_validation_data, batch_size=BATCH_SIZE, shuffle=False)
sent140_test_loader = DataLoader(sent140_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(sent140_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps)

In [ ]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"sent140", sent140_train_loader, sent140_val_loader, optimizer, scheduler, device, epochs=EPOCHS
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.")

In [ ]:
lora_model.load_state_dict(torch.load("sent140_best_model_state.bin"))
    
test_loss, test_acc, test_f1 = eval_model(lora_model, sent140_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
model_performance = []

# Funzione per memorizzare le performance sul task appena addestrato
def add_task_results(task_name, training_time, emissions, test_loss, test_acc, test_f1):
    model_performance.append({
        "Task": task_name,
        "Training Time": training_time,
        "CO2 Emissions": emissions,
        "Test Loss": test_loss,
        "Accuracy": test_acc,
        "F1 Score": test_f1,
    })

In [ ]:
# Memorizzazione dei risultati su Sentiment140
add_task_results(
    task_name="sentiment140", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## IMDB Rewiews

In [3]:
# Ottenimento del dataset
imdb_dataset = load_dataset("stanfordnlp/imdb")
print(imdb_dataset)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
from datasets import Dataset

imdb_train_data = pd.DataFrame(imdb_dataset["train"])
imdb_test_data = pd.DataFrame(imdb_dataset["test"])

imdb_data = Dataset.from_pandas(pd.concat([imdb_train_data, imdb_test_data], ignore_index=True))
imdb_data = imdb_data.shuffle(seed=42)

imdb_temp_sentences, imdb_test_sentences, imdb_temp_labels, imdb_test_labels = train_test_split(
                                                imdb_data['text'],
                                                imdb_data['label'], 
                                                test_size=0.1, 
                                                random_state=42,
                                                stratify=imdb_data['label'])

imdb_train_sentences, imdb_val_sentences, imdb_train_labels, imdb_val_labels = train_test_split(
                                                imdb_temp_sentences,
                                                imdb_temp_labels,
                                                test_size=0.1111,
                                                random_state=42,
                                                stratify=imdb_temp_labels)

print("Dimensioni dei set:")
print(f"Train: {len(imdb_train_sentences)}")
print(f"Validation: {len(imdb_val_sentences)}")
print(f"Test: {len(imdb_test_sentences)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(imdb_train_labels)}")
print(f"Validation: {Counter(imdb_val_labels)}")
print(f"Test: {Counter(imdb_test_labels)}")

Dimensioni dei set:
Train: 40000
Validation: 5000
Test: 5000

Distribuzione delle etichette:
Train: Counter({0: 20000, 1: 20000})
Validation: Counter({0: 2500, 1: 2500})
Test: Counter({1: 2500, 0: 2500})


In [ ]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
imdb_training_data = ClassificationDataset(sentences = imdb_train_sentences,
                           labels = imdb_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

imdb_validation_data = ClassificationDataset(sentences = imdb_val_sentences,
                           labels = imdb_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

imdb_test_data = ClassificationDataset(sentences = imdb_test_sentences,
                           labels = imdb_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device)

In [ ]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

In [ ]:
# Parametri principali
learning_rate = 5e-4
EPOCHS = 1
BATCH_SIZE = 32

# Creo i DataLoader
imdb_train_loader = DataLoader(imdb_training_data, batch_size=BATCH_SIZE, shuffle=True)
imdb_val_loader = DataLoader(imdb_validation_data, batch_size=BATCH_SIZE, shuffle=False)
imdb_test_loader = DataLoader(imdb_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(imdb_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 


In [ ]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"imdb", imdb_train_loader, imdb_val_loader, optimizer, scheduler, device, epochs=EPOCHS
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.")

In [ ]:
lora_model.load_state_dict(torch.load("imdb_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model, imdb_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
add_task_results(
    task_name="imdb", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## 20 News Group

In [5]:
# Ottenimento del dataset
news_dataset = load_dataset("SetFit/20_newsgroups")
print(news_dataset)

README.md:   0%|          | 0.00/734 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7532 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 7532
    })
})


In [6]:
from datasets import Dataset

news_train_data = pd.DataFrame(news_dataset["train"])
news_test_data = pd.DataFrame(news_dataset["test"])

news_data = Dataset.from_pandas(pd.concat([news_train_data, news_test_data], ignore_index=True))
news_data = news_data.shuffle(seed=42)

news_temp_sentences, news_test_sentences, news_temp_labels, news_test_labels = train_test_split(
                                                news_data['text'],
                                                news_data['label'], 
                                                test_size=0.1, 
                                                random_state=42,
                                                stratify=news_data['label'])

news_train_sentences, news_val_sentences, news_train_labels, news_val_labels = train_test_split(
                                                news_temp_sentences,
                                                news_temp_labels,
                                                test_size=0.1111,
                                                random_state=42,
                                                stratify=news_temp_labels)

print("Dimensioni dei set:")
print(f"Train: {len(news_train_sentences)}")
print(f"Validation: {len(news_val_sentences)}")
print(f"Test: {len(news_test_sentences)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(news_train_labels)}")
print(f"Validation: {Counter(news_val_labels)}")
print(f"Test: {Counter(news_test_labels)}")

Dimensioni dei set:
Train: 15076
Validation: 1885
Test: 1885

Distribuzione delle etichette:
Train: Counter({10: 799, 15: 797, 8: 796, 9: 795, 11: 793, 13: 792, 7: 792, 5: 790, 14: 789, 12: 788, 2: 788, 3: 786, 6: 780, 1: 779, 4: 771, 17: 752, 16: 728, 0: 639, 18: 620, 19: 502})
Validation: Counter({10: 100, 8: 100, 9: 100, 15: 100, 14: 99, 13: 99, 7: 99, 11: 99, 5: 99, 3: 98, 12: 98, 2: 98, 6: 97, 1: 97, 4: 96, 17: 94, 16: 91, 0: 80, 18: 78, 19: 63})
Test: Counter({8: 100, 10: 100, 15: 100, 11: 99, 9: 99, 7: 99, 13: 99, 14: 99, 5: 99, 2: 99, 6: 98, 3: 98, 12: 98, 1: 97, 4: 96, 17: 94, 16: 91, 0: 80, 18: 77, 19: 63})


In [ ]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
news_training_data = ClassificationDataset(sentences = news_train_sentences,
                           labels = news_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

news_validation_data = ClassificationDataset(sentences = news_val_sentences,
                           labels = news_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

news_test_data = ClassificationDataset(sentences = news_test_sentences,
                           labels = news_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=20)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.2,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device) 

In [ ]:
# Parametri principali
learning_rate = 5e-4
EPOCHS = 1
BATCH_SIZE = 32

# Creo i DataLoader
news_train_loader = DataLoader(news_training_data, batch_size=BATCH_SIZE, shuffle=True)
news_val_loader = DataLoader(news_validation_data, batch_size=BATCH_SIZE, shuffle=False)
news_test_loader = DataLoader(news_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(news_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 

In [ ]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"news", news_train_loader, news_val_loader, optimizer, scheduler, device, epochs=EPOCHS 
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 


In [ ]:
lora_model.load_state_dict(torch.load("news_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model, news_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
add_task_results(
    task_name="news", 
    training_time=total_time,
    emissions=emissions
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## DBpedia 14

In [5]:
# Ottenimento del dataset
dbpedia_train_dataset = pd.read_csv('/kaggle/input/dbpedia-ontology-dataset/train.csv')
dbpedia_val_test_dataset = pd.read_csv('/kaggle/input/dbpedia-ontology-dataset/test.csv')

# Costruisco il training set in modo da avere 2000 esempi per ognuna delle 14 classi
dbpedia_train_dataset = dbpedia_train_dataset.groupby("label").apply(lambda x: x.sample(n=2000, random_state=42))
dbpedia_train_dataset.reset_index(drop=True, inplace=True)

dbpedia_train_sentences, dbpedia_train_labels = dbpedia_train_dataset['content'], dbpedia_train_dataset['label']

# Divido i dati di test in test e val set
dbpedia_val_sentences, dbpedia_test_sentences, dbpedia_val_labels, dbpedia_test_labels = train_test_split(
                                                dbpedia_val_test_dataset['content'], 
                                                dbpedia_val_test_dataset['label'], 
                                                train_size=5000,
                                                test_size=5000,
                                                random_state=42,
                                                stratify=dbpedia_val_test_dataset['label']
                                            )

dbpedia_val_sentences = dbpedia_val_sentences.reset_index(drop=True)
dbpedia_val_labels = dbpedia_val_labels.reset_index(drop=True)

dbpedia_test_sentences = dbpedia_test_sentences.reset_index(drop=True)
dbpedia_test_labels = dbpedia_test_labels.reset_index(drop=True)

print("Dimensioni dei set:")
print(f"Train: {len(dbpedia_train_sentences)}")
print(f"Validation: {len(dbpedia_val_sentences)}")
print(f"Test: {len(dbpedia_test_sentences)}")

print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(dbpedia_train_labels.tolist())}")
print(f"Validation: {Counter(dbpedia_val_labels.tolist())}")
print(f"Test: {Counter(dbpedia_test_labels.tolist())}")


Dimensioni dei set:
Train: 28000
Validation: 5000
Test: 5000

Distribuzione delle etichette:
Train: Counter({0: 2000, 1: 2000, 2: 2000, 3: 2000, 4: 2000, 5: 2000, 6: 2000, 7: 2000, 8: 2000, 9: 2000, 10: 2000, 11: 2000, 12: 2000, 13: 2000})
Validation: Counter({11: 358, 9: 358, 2: 357, 7: 357, 1: 357, 10: 357, 6: 357, 3: 357, 0: 357, 13: 357, 12: 357, 5: 357, 8: 357, 4: 357})
Test: Counter({12: 358, 10: 358, 1: 357, 6: 357, 3: 357, 5: 357, 9: 357, 4: 357, 2: 357, 0: 357, 11: 357, 8: 357, 7: 357, 13: 357})


<ipython-input-5-db0f45ac6817>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dbpedia_train_dataset = dbpedia_train_dataset.groupby("label").apply(lambda x: x.sample(n=2000, random_state=42))


In [ ]:
MAX_SEQ_LEN = 512
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
dbpedia_training_data = ClassificationDataset(sentences = dbpedia_train_sentences,
                           labels = dbpedia_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

dbpedia_validation_data = ClassificationDataset(sentences = dbpedia_val_sentences,
                           labels = dbpedia_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

dbpedia_test_data = ClassificationDataset(sentences = dbpedia_test_sentences,
                           labels = dbpedia_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=14)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.2,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device) 

In [ ]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

for name, param in lora_model.named_parameters():
    if "classifier" in name:
        print(f"{name}: requires_grad = {param.requires_grad}")

In [ ]:
# Parametri principali
learning_rate = 5e-4
EPOCHS = 1
BATCH_SIZE = 32

# Creo i DataLoader
dbpedia_train_loader = DataLoader(dbpedia_training_data, batch_size=BATCH_SIZE, shuffle=True)
dbpedia_val_loader = DataLoader(dbpedia_validation_data, batch_size=BATCH_SIZE, shuffle=False)
dbpedia_test_loader = DataLoader(dbpedia_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(dbpedia_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 


In [ ]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"dbpedia", dbpedia_train_loader, dbpedia_val_loader, optimizer, scheduler, device, epochs=EPOCHS 
)
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 

In [ ]:
lora_model.load_state_dict(torch.load("dbpedia_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model, dbpedia_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
add_task_results(
    task_name="dbpedia", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## Emotion Dataset

In [7]:
# Ottenimento del dataset
emotion_dataset = load_dataset("dair-ai/emotion")
print(emotion_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [8]:
#Divido i dati in training, validation e test set
emotion_train_data = emotion_dataset["train"].shuffle(seed=42)
emotion_val_data = emotion_dataset["validation"].shuffle(seed=42)
emotion_test_data = emotion_dataset["test"].shuffle(seed=42)

emotion_train_sentences, emotion_train_labels = emotion_train_data['text'],emotion_train_data['label']
emotion_val_sentences, emotion_val_labels = emotion_val_data['text'],emotion_val_data['label']
emotion_test_sentences, emotion_test_labels = emotion_test_data['text'],emotion_test_data['label']

print("Dimensioni dei set:")
print(f"Train: {len(emotion_train_sentences)}")
print(f"Validation: {len(emotion_val_sentences)}")
print(f"Test: {len(emotion_test_sentences)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(emotion_train_labels)}")
print(f"Validation: {Counter(emotion_val_labels)}")
print(f"Test: {Counter(emotion_test_labels)}")

Dimensioni dei set:
Train: 16000
Validation: 2000
Test: 2000

Distribuzione delle etichette:
Train: Counter({1: 5362, 0: 4666, 3: 2159, 4: 1937, 2: 1304, 5: 572})
Validation: Counter({1: 704, 0: 550, 3: 275, 4: 212, 2: 178, 5: 81})
Test: Counter({1: 695, 0: 581, 3: 275, 4: 224, 2: 159, 5: 66})


In [11]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
emotion_training_data = ClassificationDataset(
                            sentences = emotion_train_sentences,
                            labels = emotion_train_labels,
                            tokenizer = tokenizer,
                            max_len = MAX_SEQ_LEN)

emotion_validation_data = ClassificationDataset(
                            sentences = emotion_val_sentences,
                            labels = emotion_val_labels,
                            tokenizer = tokenizer,
                            max_len = MAX_SEQ_LEN)

emotion_test_data = ClassificationDataset(
                            sentences = emotion_test_sentences,
                            labels = emotion_test_labels,
                            tokenizer = tokenizer,
                            max_len = MAX_SEQ_LEN)

In [12]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.3,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,769,472 || all params: 111,256,326 || trainable%: 1.5904


PeftModel(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.3, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768

In [13]:
# Parametri principali
learning_rate = 2e-4
EPOCHS = 10
BATCH_SIZE = 32

# Creo i DataLoader
emotion_train_loader = DataLoader(emotion_training_data, batch_size=BATCH_SIZE, shuffle=True)
emotion_val_loader = DataLoader(emotion_validation_data, batch_size=BATCH_SIZE, shuffle=False)
emotion_test_loader = DataLoader(emotion_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(emotion_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 

In [14]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

In [15]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"emotion", emotion_train_loader, emotion_val_loader, optimizer, scheduler, device, epochs=EPOCHS 
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 

[codecarbon INFO @ 09:27:05] [setup] RAM Tracking...
[codecarbon INFO @ 09:27:05] [setup] CPU Tracking...
[codecarbon INFO @ 09:27:05] Tracking Intel CPU via RAPL interface
[codecarbon INFO @ 09:27:06] [setup] GPU Tracking...
[codecarbon INFO @ 09:27:06] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:27:06] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 09:27:06] GPU number 7 will not be monitored, at your request.
[codecarbon WARNING @ 09:27:06] You have 8 GPUs but we will monitor only 1 of them. Check your configuration.
[codecarbon INFO @ 09:27:06] >


Epoch 1/10


Training  :  14%|█▍        | 71/500 [00:14<01:27,  4.88it/s, accuracy=0.338, loss=1.57][codecarbon INFO @ 09:27:24] Energy consumed for RAM : 0.001183 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 09:27:24] Energy consumed for all CPUs : 0.000351 kWh. Total CPU Power : 83.96225757274186 W
[codecarbon INFO @ 09:27:24] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 09:27:24] Energy consumed for all GPUs : 0.000607 kWh. Total GPU Power : 145.21206471849285 W
[codecarbon I


Epoch 2/10


Training  :   8%|▊         | 40/500 [00:08<01:35,  4.84it/s, accuracy=0.896, loss=0.524][codecarbon INFO @ 09:29:09] Energy consumed for RAM : 0.009406 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 09:29:09] Energy consumed for all CPUs : 0.002790 kWh. Total CPU Power : 84.46513782203752 W
[codecarbon INFO @ 09:29:09] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 09:29:09] GPU number 7 will not be monitored, at your request.
Training  :   8%|▊         | 41/500 [00:08<01:34,  4.84it/s, accuracy=0.894, loss=0.524][codecarbon INFO @ 09:29:09] Energy c

...


Epoch 9/10


Training  :   2%|▏         | 10/500 [00:02<01:41,  4.81it/s, accuracy=0.969, loss=0.266][codecarbon INFO @ 09:42:09] Energy consumed for RAM : 0.070486 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 09:42:09] Energy consumed for all CPUs : 0.020854 kWh. Total CPU Power : 84.03199494985459 W
[codecarbon INFO @ 09:42:09] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 09:42:09] Energy consumed for all GPUs : 0.038655 kWh. Total GPU Power : 152.62313096088272 W
[codecarbon 

La loss sul validation set non è migliorata per 1 epoche.

Epoch 10/10


Training  :  10%|▉         | 49/500 [00:10<01:33,  4.81it/s, accuracy=0.956, loss=0.271][codecarbon INFO @ 09:44:09] Energy consumed for RAM : 0.079884 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 09:44:09] Energy consumed for all CPUs : 0.023629 kWh. Total CPU Power : 83.61752241205264 W
[codecarbon INFO @ 09:44:09] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 09:44:09] Energy consumed for all GPUs : 0.043841 kWh. Total GPU Power : 156.03840347836447 W
[codecarbon 

La loss sul validation set non è migliorata per 2 epoche.

Emissioni CO₂ totali: 0.0536 kg

BERT with LoRA Training Time: 1121.75 seconds, 18.70 minutes.


In [17]:
lora_model.load_state_dict(torch.load("emotion_best_model_state.bin"))
 
test_loss, test_acc, test_f1 = eval_model(lora_model, emotion_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

Evaluating: 100%|██████████| 63/63 [00:06<00:00,  9.27it/s, accuracy=0.929, loss=0.284]

LoRA Fine-Tuning - Test loss: 0.2841, Accuracy: 0.9295, F1 score: 0.8848


In [ ]:
add_task_results(
    task_name="emotion", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## RTE

In [4]:
# Ottenimento del dataset
rte_dataset = load_dataset("glue", "rte")
print(rte_dataset)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})


In [5]:
from datasets import Dataset

rte_train_data = pd.DataFrame(rte_dataset["train"])
rte_val_data = pd.DataFrame(rte_dataset["validation"])

rte_data = Dataset.from_pandas(pd.concat([rte_train_data, rte_val_data], ignore_index=True))
rte_data = rte_data.shuffle(seed=42)

rte_temp_sentences1, rte_test_sentences1, rte_temp_sentences2, rte_test_sentences2,  rte_temp_labels, rte_test_labels = train_test_split(
                                                rte_data['sentence1'],
                                                rte_data['sentence2'],
                                                rte_data['label'], 
                                                test_size=0.1, 
                                                random_state=42,
                                                stratify=rte_data['label'])

rte_train_sentences1, rte_val_sentences1, rte_train_sentences2, rte_val_sentences2, rte_train_labels, rte_val_labels = train_test_split(
                                                rte_temp_sentences1,
                                                rte_temp_sentences2,
                                                rte_temp_labels,
                                                test_size=0.1111,
                                                random_state=42,
                                                stratify=rte_temp_labels)

print(f"Train: {len(rte_train_sentences1)}")
print(f"Validation: {len(rte_val_sentences1)}")
print(f"Test: {len(rte_test_sentences1)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(rte_train_labels)}")
print(f"Validation: {Counter(rte_val_labels)}")
print(f"Test: {Counter(rte_test_labels)}")

Train: 2213
Validation: 277
Test: 277

Distribuzione delle etichette:
Train: Counter({0: 1115, 1: 1098})
Validation: Counter({0: 140, 1: 137})
Test: Counter({0: 140, 1: 137})


In [ ]:
from torch.utils.data import Dataset

class NLIDataset(Dataset):

    def __init__(self, sentences1, sentences2 , labels, tokenizer, max_len):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.sentences1)
    
    def __getitem__(self,index):
        sentence1 = self.sentences1[index]
        sentence2 = self.sentences2[index]
        label = self.labels[index]
        
        encoding = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=True,
            padding="max_length",
            return_attention_mask=True,
            return_tensors='pt')
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding["token_type_ids"].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
            }

In [ ]:
MAX_SEQ_LEN = 256
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
rte_training_data = NLIDataset(
                           sentences1 = rte_train_sentences1,
                           sentences2 = rte_train_sentences2,
                           labels = rte_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

rte_validation_data = NLIDataset(
                           sentences1 = rte_val_sentences1,
                           sentences2 = rte_val_sentences2,
                           labels = rte_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

rte_test_data = NLIDataset(
                           sentences1 = rte_test_sentences1,
                           sentences2 = rte_test_sentences2,
                           labels = rte_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=128,
    lora_dropout=0.2,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device) 

In [ ]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

In [ ]:
# Parametri principali
learning_rate = 2e-4
EPOCHS = 1
BATCH_SIZE = 32

# Creo i DataLoader
rte_train_loader = DataLoader(rte_training_data, batch_size=BATCH_SIZE, shuffle=True)
rte_val_loader = DataLoader(rte_validation_data, batch_size=BATCH_SIZE, shuffle=False)
rte_test_loader = DataLoader(rte_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(rte_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 

In [ ]:
history, total_time = train_and_evaluate_model(
    lora_model,"rte", rte_train_loader, rte_val_loader, optimizer, scheduler, device, apochs= EPOCHS
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 

In [ ]:
lora_model.load_state_dict(torch.load("rte_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model, rte_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
add_task_results(
    task_name="rte", 
    training_time=total_time,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## QQP

In [7]:
# Ottenimento del dataset
qqp_dataset = load_dataset("glue", "qqp")
print(qqp_dataset)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 390965
    })
})


In [8]:
from datasets import Dataset

from collections import Counter
from datasets import Dataset

def balance_dataset(dataset, num_example):
    class_0 = [example for example in dataset if example["label"] == 0][:num_example]
    class_1 = [example for example in dataset if example["label"] == 1][:num_example]

    balanced_data = class_0 + class_1

    balanced_dataset = Dataset.from_list(balanced_data)
    balanced_dataset = balanced_dataset.shuffle(seed=42)

    return balanced_dataset 
 


def get_val_test_set(dataset, val_size, test_size):
    class_0 = [example for example in dataset if example["label"] == 0]
    class_1 = [example for example in dataset if example["label"] == 1]    

    val_set = class_0[:val_size//2] + class_1[:val_size//2]
    test_set = class_0[val_size//2:val_size//2 + test_size//2] + class_1[val_size//2:val_size//2 + test_size//2]

    val_set = Dataset.from_dict({k: [example[k] for example in val_set] for k in val_set[0]})
    test_set = Dataset.from_dict({k: [example[k] for example in test_set] for k in test_set[0]})
    
    val_set = val_set.shuffle(seed=42)
    test_set = test_set.shuffle(seed=42)

    return val_set, test_set


qqp_train_data = qqp_dataset["train"]
qqp_val_test_data = qqp_dataset["validation"]

qqp_train_data = balance_dataset(qqp_train_data,10000)
qqp_val_data, qqp_test_data  =  get_val_test_set(qqp_val_test_data, 5000, 5000)

qqp_train_questions1, qqp_train_questions2, qqp_train_labels = qqp_train_data['question1'],  qqp_train_data['question2'], qqp_train_data['label']
qqp_val_questions1, qqp_val_questions2, qqp_val_labels = qqp_val_data['question1'],  qqp_val_data['question2'], qqp_val_data['label']
qqp_test_questions1, qqp_test_questions2, qqp_test_labels = qqp_test_data['question1'],  qqp_test_data['question2'], qqp_test_data['label']


print("Dimensioni dei set:")
print(f"Train: {len(qqp_train_questions1)}")
print(f"Validation: {len(qqp_val_questions1)}")
print(f"Test: {len(qqp_test_questions1)}")

print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(qqp_train_labels)}")
print(f"Validation: {Counter(qqp_val_labels)}")
print(f"Test: {Counter(qqp_test_labels)}")

Dimensioni dei set:
Train: 20000
Validation: 5000
Test: 5000

Distribuzione delle etichette:
Train: Counter({1: 10000, 0: 10000})
Validation: Counter({0: 2500, 1: 2500})
Test: Counter({0: 2500, 1: 2500})


In [ ]:
MAX_SEQ_LEN = 256
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
qqp_training_data = NLIDataset(
                           sentences1 = qqp_train_questions1,
                           sentences2 = qqp_train_questions2,
                           labels = qqp_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

qqp_validation_data = NLIDataset(
                           sentences1 = qqp_val_questions1,
                           sentences2 = qqp_val_questions2,
                           labels = qqp_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

qqp_test_data = NLIDataset(
                           sentences1 = qqp_test_questions1,
                           sentences2 = qqp_test_questions2,
                           labels = qqp_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=128,
    lora_dropout=0.3,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device) 

In [ ]:
for name, param in lora_model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True

In [ ]:
# Parametri principali
learning_rate = 5e-5
EPOCHS = 1
BATCH_SIZE = 32

# Creo i DataLoader
qqp_train_loader = DataLoader(qqp_training_data, batch_size=BATCH_SIZE, shuffle=True)
qqp_val_loader = DataLoader(qqp_validation_data, batch_size=BATCH_SIZE, shuffle=False)
qqp_test_loader = DataLoader(qqp_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len(qqp_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps)

In [ ]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model,"qqp", qqp_train_loader, qqp_val_loader, optimizer, scheduler, device, epochs=EPOCHS 
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 

In [ ]:
lora_model.load_state_dict(torch.load("qqp_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model, qqp_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

In [ ]:
add_task_results(
    task_name="qqp", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)

## COLA

In [18]:
# Ottenimento del dataset
cola_dataset = load_dataset("glue", "cola")
print(cola_dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})


In [19]:
from datasets import Dataset

cola_train_data = pd.DataFrame(cola_dataset["train"])
cola_val_data = pd.DataFrame(cola_dataset["validation"])

cola_data = Dataset.from_pandas(pd.concat([cola_train_data, cola_val_data], ignore_index=True))
cola_data = cola_data.shuffle(seed=42)

cola_temp_sentences, cola_test_sentences, cola_temp_labels, cola_test_labels = train_test_split(
                                                cola_data['sentence'],
                                                cola_data['label'], 
                                                test_size=0.1, 
                                                random_state=42,
                                                stratify=cola_data['label'])

cola_train_sentences, cola_val_sentences, cola_train_labels, cola_val_labels = train_test_split(
                                                cola_temp_sentences, 
                                                cola_temp_labels,
                                                test_size=0.1111,
                                                random_state=42,
                                                stratify=cola_temp_labels)

print("Dimensioni dei set:")
print(f"Train: {len(cola_train_sentences)}")
print(f"Validation: {len(cola_val_sentences)}")
print(f"Test: {len(cola_test_sentences)}")

# Verifica distribuzione delle etichette
print("\nDistribuzione delle etichette:")
print(f"Train: {Counter(cola_train_labels)}")
print(f"Validation: {Counter(cola_val_labels)}")
print(f"Test: {Counter(cola_test_labels)}")

Dimensioni dei set:
Train: 7674
Validation: 960
Test: 960

Distribuzione delle etichette:
Train: Counter({1: 5394, 0: 2280})
Validation: Counter({1: 675, 0: 285})
Test: Counter({1: 675, 0: 285})


In [20]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 32

# Inizializzo il Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ottiengo i dataset
cola_training_data = ClassificationDataset(sentences = cola_train_sentences,
                           labels = cola_train_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

cola_validation_data = ClassificationDataset(sentences = cola_val_sentences,
                           labels = cola_val_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

cola_test_data = ClassificationDataset(sentences = cola_test_sentences,
                           labels = cola_test_labels,
                           tokenizer = tokenizer,
                           max_len = MAX_SEQ_LEN)

In [21]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Pretrained model
lora_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=128,
    lora_dropout=0.3,
    target_modules=["query", "key", "value"],  
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()
lora_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 884,736 || all params: 110,368,514 || trainable%: 0.8016


PeftModel(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.3, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768

In [25]:
# Parametri principali
learning_rate = 5e-5
EPOCHS = 10
BATCH_SIZE = 32

# Creo i DataLoader
cola_train_loader = DataLoader( cola_training_data, batch_size=BATCH_SIZE, shuffle=True)
cola_val_loader = DataLoader( cola_validation_data, batch_size=BATCH_SIZE, shuffle=False)
cola_test_loader = DataLoader( cola_test_data, batch_size=BATCH_SIZE, shuffle=False)

total_steps = len( cola_train_loader) * EPOCHS

# Ottimizzatore
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr = learning_rate)


# Scheduler
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                       num_warmup_steps = 0,
                                                       num_training_steps = total_steps) 

In [26]:
history, total_time, emissions = train_and_evaluate_model(
    lora_model," cola",  cola_train_loader,  cola_val_loader, optimizer, scheduler, device, epochs=EPOCHS 
) 
print(f"\nBERT with LoRA Training Time: {total_time:.2f} seconds, {total_time/60:.2f} minutes.") 

[codecarbon INFO @ 10:30:22] [setup] RAM Tracking...
[codecarbon INFO @ 10:30:22] [setup] CPU Tracking...
[codecarbon INFO @ 10:30:22] Tracking Intel CPU via RAPL interface
[codecarbon INFO @ 10:30:23] [setup] GPU Tracking...
[codecarbon INFO @ 10:30:23] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:30:23] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 10:30:23] GPU number 7 will not be monitored, at your request.
[codecarbon WARNING @ 10:30:23] You have 8 GPUs but we will monitor only 1 of them. Check your configuration.
[codecarbon INFO @ 10:30:23] >


Epoch 1/10


Training  :  31%|███       | 74/240 [00:14<00:33,  4.97it/s, accuracy=0.733, loss=0.531][codecarbon INFO @ 10:30:41] Energy consumed for RAM : 0.001181 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 10:30:41] Energy consumed for all CPUs : 0.000344 kWh. Total CPU Power : 82.26544695669658 W
[codecarbon INFO @ 10:30:41] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 10:30:41] Energy consumed for all GPUs : 0.000623 kWh. Total GPU Power : 149.13961766893053 W
[codecarbon 


Epoch 2/10


Training  :  17%|█▋        | 40/240 [00:08<00:40,  4.98it/s, accuracy=0.802, loss=0.459][codecarbon INFO @ 10:31:26] Energy consumed for RAM : 0.004705 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 10:31:26] Energy consumed for all CPUs : 0.001372 kWh. Total CPU Power : 82.54307345029055 W
[codecarbon INFO @ 10:31:26] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 10:31:26] Energy consumed for all GPUs : 0.002486 kWh. Total GPU Power : 149.677021162879 W
[codecarbon IN

...

La loss sul validation set non è migliorata per 1 epoche.

Epoch 9/10


Training  :   3%|▎         | 7/240 [00:01<00:47,  4.94it/s, accuracy=0.848, loss=0.329][codecarbon INFO @ 10:37:26] Energy consumed for RAM : 0.032900 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 10:37:26] Energy consumed for all CPUs : 0.009654 kWh. Total CPU Power : 83.29834615485152 W
[codecarbon INFO @ 10:37:26] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 10:37:26] Energy consumed for all GPUs : 0.017960 kWh. Total GPU Power : 157.25899491092224 W
[codecarbon I

La loss sul validation set non è migliorata per 2 epoche.

Epoch 10/10


Training  :  19%|█▉        | 46/240 [00:09<00:39,  4.93it/s, accuracy=0.867, loss=0.331][codecarbon INFO @ 10:38:26] Energy consumed for RAM : 0.037600 kWh. RAM Power : 282.8781752586365 W
[codecarbon INFO @ 10:38:26] Energy consumed for all CPUs : 0.011036 kWh. Total CPU Power : 83.37826227118813 W
[codecarbon INFO @ 10:38:26] GPU number 0 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 1 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 4 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 5 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] GPU number 7 will not be monitored, at your request.
[codecarbon INFO @ 10:38:26] Energy consumed for all GPUs : 0.020579 kWh. Total GPU Power : 159.18872128308843 W
[codecarbon 

La loss sul validation set non è migliorata per 3 epoche.
Early stopping attivato dopo 3 epoche senza miglioramenti

Emissioni CO₂ totali: 0.0249 kg

BERT with LoRA Training Time: 522.73 seconds, 8.71 minutes.


In [27]:
lora_model.load_state_dict(torch.load(" cola_best_model_state.bin"))
test_loss, test_acc, test_f1 = eval_model(lora_model,  cola_test_loader, device)
print(f"LoRA Fine-Tuning - Test loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 score: {test_f1:.4f}")

Evaluating: 100%|██████████| 30/30 [00:03<00:00,  9.53it/s, accuracy=0.815, loss=0.441]

LoRA Fine-Tuning - Test loss: 0.4411, Accuracy: 0.8146, F1 score: 0.7593


In [ ]:
add_task_results(
    task_name="cola", 
    training_time=total_time,
    emissions=emissions,
    test_loss=test_loss,
    test_acc=test_acc,
    test_f1=test_f1,
)

performance = pd.DataFrame(model_performance)
print(performance)